In [ ]:
# =========================================================================
# The Corporate Knowledge Navigator: Complete Multi-Agent Setup
# Project: Enterprise Agent for RAG and System Status Routing
# =========================================================================

# --- 1. Installation of Required Libraries ---
print("--- 1. Installing required libraries (Wait for this to complete) ---")
# Installing all necessary packages robustly, including community and text-splitters
!pip install -q langchain-google-genai pypdf chromadb langchain langchain-community langchain-text-splitters
print("Installation Complete.")

# --- 2. API Key and LLM Setup ---
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenAIEmbeddings
from langchain.tools import tool # Required for Custom Tool

# **IMPORTANT**: Your provided API Key is set here.
# WARNING: Do not share your API Key publicly in your final submission.
os.environ["GEMINI_API_KEY"] = "YOUR_API_KEY"

# Initialize the LLM for the Agent and Embeddings for RAG
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
print("Gemini LLM initialized.")

# --- 3. RAG (Memory Bank) Setup: Building the Knowledge Base ---
print("--- 3. Setting up RAG (Memory Bank) ---")
# Using the stable 'community' and robust imports for LangChain components
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.agents import create_retriever_tool

try:
    # Load and split the uploaded PDF
    loader = PyPDFLoader("Policies.pdf")
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = text_splitter.split_documents(documents)
    print(f"Total Chunks created: {len(docs)}")

    # Create Embeddings and Vector Store
    embedding_function = GoogleGenAIEmbeddings(model="text-embedding-004")
    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=embedding_function,
        collection_name="corporate_policies_db"
    )

    # Create the RAG Retriever Tool
    policy_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    retriever_tool = create_retriever_tool(
        policy_retriever,
        "corporate_policy_retriever",
        "Use this tool ONLY for corporate document lookups, HR policies, and financial memos. Do NOT use this for IT system status."
    )
    print("SUCCESS: RAG Memory Bank and Retriever Tool Ready.")

except Exception as e:
    print(f"FATAL ERROR IN RAG SETUP. Please check 'Policies.pdf' upload and API Key. Error: {e}")
    # If this fails, the user must manually check the file upload.


# --- 4. Custom Tool Definition: Simulating External API ---
print("--- 4. Defining Custom Tool (IT Status Checker) ---")

@tool
def check_internal_system_status(query: str) -> str:
    """
    Use this tool ONLY to check the status of internal IT systems or applications,
    such as VPN, Email server, or JIRA. This is the official system status checker.
    Example questions: 'Is the VPN server down?' or 'Check Email status'.
    """
    # Logic to simulate checking an external status API
    if "VPN" in query.upper() or "VPN SERVER" in query.upper():
        return "VPN Server Status: Operational, but high load is expected during 9 AM to 11 AM."
    elif "EMAIL" in query.upper() or "EMAIL SERVER" in query.upper():
        return "Email Server Status: Operational and stable."
    else:
        return "System Status Checker: Please specify the system (e.g., VPN, Email). Could not find status for the specified system."

print("Custom Tool defined.")


# --- 5. Master Agent (Router) Setup and Agent Executor ---
print("--- 5. Setting up Master Agent (Router) ---")
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool as tool_decorator

# List of all available Tools (RAG and Custom Tool)
all_tools = [check_internal_system_status, retriever_tool]

# Prompt for the Router/Master Agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are the Corporate Knowledge Navigator, a master agent and expert corporate assistant. Your goal is to answer employee queries accurately. You MUST analyze the user's query and decide which tool (corporate_policy_retriever for documents, or check_internal_system_status for IT status) to use. If no tool is required, answer directly."),
    ("placeholder", "{agent_scratchpad}"),
    ("human", "{input}"),
])

# Create the Agent Executor (This handles the Routing, Tool Use, and Final Answer)
master_agent = create_tool_calling_agent(llm, all_tools, prompt)
agent_executor = AgentExecutor(
    agent=master_agent,
    tools=all_tools,
    verbose=True # ESSENTIAL for Observability and high scores
)
print("Master Agent (Router) set up and ready for testing.")


# --- 6. Final Test Examples (For Submission Demo) ---
# Running the tests to show successful routing and RAG retrieval

print("\n\n========================================================")
print("TESTING AGENT ROUTING AND RETRIEVAL (Observe 'Thought' process)")
print("========================================================\n")

# TEST 1: Policy Question (Should use RAG Retriever Tool)
print("\n--- TEST 1: Policy Question (RAG) ---")
policy_query = "What is the policy for submitting paid leave requests?"
response_policy = agent_executor.invoke({"input": policy_query})
print("\nFinal Agent Answer:")
print(response_policy["output"])


# TEST 2: IT Status Question (Should use Custom Tool)
print("\n--- TEST 2: IT Status Question (Custom Tool) ---")
status_query = "Is the VPN server currently experiencing any high load?"
response_status = agent_executor.invoke({"input": status_query})
print("\nFinal Agent Answer:")
print(response_status["output"])